# Spectral Simulations with BR and GCS
***
auth: Matthew Brown

date: 2023.04.20

edited: Matt Prilliman

date: 2024.09.27
***
For spectral simulations, we use the Generate Cumulative Sky (GCS) model. GCS is very picky about the weather file used and it must have exactly 8760 data entries (1 year). See the file `weather_resample.ipynb` to convert a single month TMY file into a full year.

The steps are as follows:
1. Prepare Data
    - Get a TMY-like weather file. If it is only a month, use `weather_resample.ipynb` to pad the data with zeros so there is a full year.
    - Choose an appropriate ground material. For a help with this, or to see a complete list, open `selecting_albedo.ipynb`. Examples: Grass, LiteSoil, Concrete, Snow
2. Generate the spectra files
3. Generate spectral TMYs
4. Set simulation temporal, spatial, wavelength resolution
5. Simulate
6. Compile Simulation Results

In [1]:
import os
import re
from datetime import datetime as dt
from datetime import date
import bifacial_radiance as br
import numpy as np
import pandas as pd
from itertools import product

In [2]:
# This information helps with debugging and getting support :)
import sys, platform
print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("Pandas version ", pd.__version__)
print("bifacial_radiance version ", br.__version__)

Working on a  Windows 10
Python version  3.11.7 | packaged by Anaconda, Inc. | (main, Dec 15 2023, 18:05:47) [MSC v.1916 64 bit (AMD64)]
Pandas version  2.1.4
bifacial_radiance version  0.1.dev1951+g7750ba7.d20240906


## 1. Prepare

- Create Radiance Object
- Read weather

In [3]:
MET_FILE = r'demo_tmy3_fixed.csv'

SPECTRAL_TMY_FOLDER = r'data/spectral_tmys'

demo = br.RadianceObj('demo')

# you MUST coerce year
metdata = demo.readWeatherFile(MET_FILE, coerce_year=2021)

path = C:\Users\mprillim\sam_dev\bifacial_radiance\docs\development
Making path: images
Making path: objects
Making path: results
Making path: skies
Making path: EPWs
Making path: materials
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2021
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos


C:\Users\mprillim\AppData\Local\anaconda3\Lib\site-packages\pvlib\iotools\tmy.py:201: DtypeWarning: Columns (14,17,20,23,26,29,32,35,38,41,44,47,50,53,56,59,62,66) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(fbuf, header=0)


## 2. Generate Spectra Files

This will use pySMARTS to generate spectral irradiance files for DNI, DHI, GHI, and ALB.
Different Ground materials are available.

In [4]:
cwd = os.getcwd()
alb, dni, dhi, ghi = demo.generate_spectra(ground_material='Grass')
os.chdir(cwd)

 -=   Spectral Simulation   =- 
 Spectra files will be saved.


Generating Spectra: 100%|█████████████████████████████████████████| 493/493 [02:44<00:00,  2.99it/s]


## 3. Generate Spectral TMYs

This creates TMY-like weather files with all the original metadata of the source weather file. 
Instead of total irradiance, each file only stores the irradiance of a single wavelength. Thus,
there is one file per wavelength tested.

In [6]:
location_name = 'CO_Golden'
wavelengths = np.arange(300,401,25)
demo.generate_spectral_tmys(wavelengths=wavelengths, weather_file=MET_FILE, spectra_folder=r'spectra', location_name=location_name)

C:\Users\mprillim\AppData\Local\anaconda3\Lib\site-packages\pvlib\iotools\tmy.py:201: DtypeWarning: Columns (14,17,20,23,26,29,32,35,38,41,44,47,50,53,56,59,62,66) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(fbuf, header=0)
C:\Users\mprillim\AppData\Local\anaconda3\Lib\site-packages\pvlib\iotools\tmy.py:247: pvlibDeprecationWarning: TMY3 variable names will be renamed to pvlib conventions by default starting in pvlib 0.11.0. Specify map_variables=True to enable that behavior now, or specify map_variables=False to hide this warning.
  warnings.warn(
Generating Spectral TMYs: 100%|███████████████████████████████████████| 5/5 [00:01<00:00,  2.74it/s]


In [6]:
print(spectral_df)

None


## 4. Build Simulation Parameters

You will need to run several simulations based on the resolution you need.

- **Time:** Year, Month, Days, and Hours. Ranges above 1 day should be done with an HPC
- **Wavelengths:** A list or array of discrete, integer wavelengths. More than a handful should be done with an HPC
- **Test Points:** rows and modules within the array to test. Again, more than 2 or 3 of each should be done with an HPC

Then, build a simulation list from the time and wavelength components.

In [7]:
# select year, month, days, hours
year = 2021
month = 7
days = range(2,3)  # NOTE: range end-point is not inclusive
hours = range(10,12) # NOTE: see above
location_name = "CO_Golden"
wavelengths = np.arange(300,401,25)

# select Rows, Modules, Ground Type
rows = [1,2]
mods = [4]
ground_material = 'Grass'

# select root directory for simulation data
file_date = dt.strftime(date.today(), "%d%m%y")
rootPath = r'C:\Users\mprillim\Documents\bifacial_radiance\original\spectral_radiance_tutorial'

# select directory containing weather data
met_path = r'C:\Users\mprillim\Documents\bifacial_radiance\original\spectral_radiance_tutorial\data\spectral_tmys'

# built a simulation list
dt_list = [f'{year}-{month:02}-{d:02}_{h:02}00' for d in days for h in hours]
#dt_list = pd.to_datetime(dt_list, format='%Y-%m-%d_%H%M')
sim_list = pd.DataFrame(product(dt_list,wavelengths), columns=['datetime','wavelength'])
sim_list.head()
print(sim_list)

          datetime  wavelength
0  2021-07-02_1000         300
1  2021-07-02_1000         325
2  2021-07-02_1000         350
3  2021-07-02_1000         375
4  2021-07-02_1000         400
5  2021-07-02_1100         300
6  2021-07-02_1100         325
7  2021-07-02_1100         350
8  2021-07-02_1100         375
9  2021-07-02_1100         400


In [8]:
def spectral_gcs(wavelength, date_time):
    
    labels = re.split('-|_', date_time)
    labels[-1] = labels[-1][:2]
    labels = [int(j) for j in labels]
    met_file = os.path.join(met_path,f'{location_name}_TMY_w{wavelength:04}.csv')
    sim_path = os.path.join(rootPath,'results',f'{ground_material}',f'd_{labels[2]:02}',f'h_{labels[3]:02}',f'w_{wavelength:04}')    
    if not os.path.exists(sim_path):
        os.makedirs(sim_path, exist_ok=True)
    
    rad_obj = br.RadianceObj(name='demo',path=sim_path)
    met_data = rad_obj.readWeatherFile(met_file, coerce_year=2021,
                                       starttime=date_time, endtime=date_time)    
    print(met_data.albedo)
    tilt = round(rad_obj.getSingleTimestampTrackerAngle(timeindex=0, gcr=0.35, limit_angle=50),1)
    rad_obj.setGround(met_data.albedo)
    trackerdict = rad_obj.genCumSky()

    scene_dict = {'tilt': tilt,
                 'gcr':0.35,
                 'hub_height': 1.5,
                 'nMods': 10,
                 'nRows': 4}
    scene = rad_obj.makeScene(module="PrismSolar-Bi60", sceneDict=scene_dict)

    oct_file = rad_obj.makeOct()

    for r in rows:
        for m in mods:
            cname = f'd{labels[2]}_h{labels[3]}_w{wavelength:04}'
            analysis = br.AnalysisObj(octfile=oct_file)
            frontscan, backscan = analysis.moduleAnalysis(scene=scene, sensorsy=6,
                                                        modWanted=m, rowWanted=r,)
            frontdict, backdict = analysis.analysis(octfile=oct_file, name=cname,
                                                    frontscan=frontscan,
                                                    backscan=backscan)


## 5. Run The Simulations

The following example is as follows:
- 1 Day
- 4 Hours
- 5 Wavelengths
- 2 Points in the Array

Thats 20 simulations with 2 calculations each. Thats 40 result files.

executions time on my machine ~15 minutes


In [9]:
for i in sim_list.index:
    wavelength = sim_list.loc[i,'wavelength']
    datetime =   sim_list.loc[i,'datetime']
    spectral_gcs(wavelength,datetime)
os.chdir(rootPath)


path = C:\Users\mprillim\Documents\bifacial_radiance\original\spectral_radiance_tutorial\results\Grass\d_02\h_10\w_0300
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2021
Filtering dates
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos
[0.0336]
Loading albedo, 1 value(s), 0.034 avg
1 nonzero albedo values.
Loaded  EPWs\metdata_temp.csv
message: 'gencumulativesky' is not recognized as an internal or external command,
operable program or batch file.
Created demo.oct
Linescan in process: d2_h10_w0300_Row1_Module4_Front
message: rtrace: skybright`cumulative: undefined variable
Linescan in process: d2_h10_w0300_Row1_Module4_Back
message: rtrace: skybright`cumulative: undefined variable
Linescan in process: d2_h10_w0300_Row2_Module4_Front
message: r

## 6. Compile Simulation Results

The compiler will walk through the simulation directory and organize your results into a single file. If it detects any missing data, it will generate an additional file with the indices of any missing simulation. This is not likely to happen on a local machine, and is more useful for multi-threaded environments.

In [10]:
def _buffer(path):
    '''
    generate int and str list of directories
    '''
    dirs = next(os.walk(path))[1]
    dirs.sort()
    dirlist = [int(i[2:]) for i in dirs]
    return dirs, dirlist

def _rowmod(resFile):
    '''
    pull row and mod integer from filename
    '''
    resFile = resFile[:-4]
    splat = resFile.split('_')
    r = int(splat[-2][3:])
    m = int(splat[-1][6:])
    return r, m

def _checkForMissing(idx,sim_list,rows,modules):
    """
    looks for missing data (by matrix of iterables)
    """

    print('Looking for missing data...')

    cols = ['datetime','row','module','wavelength']

    # -- what we want
    wdates = np.unique(sim_list.datetime.to_numpy())
    wwaves = np.unique(sim_list.wavelength.to_numpy())    
    want = pd.DataFrame(product(wdates,rows,modules,wwaves), columns=cols)

    # -- what we have compiled
    hdates = np.unique(idx.get_level_values('datetime').to_numpy())
    hrows = np.unique(idx.get_level_values('row').to_numpy())
    hmods = np.unique(idx.get_level_values('mod').to_numpy())
    hwaves = np.unique(idx.get_level_values('wavelength').to_numpy())
    have = pd.DataFrame(product(hdates,hrows,hmods,hwaves), columns=cols)

    try:
        pd.testing.assert_frame_equal(have, want)
        print("No results missing.")
        return None
    except:
        print("Results Missing... ")

    # -- what is missing
    keys = list(want.columns.values)
    idhave = have.set_index(keys).index
    idwant = want.set_index(keys).index
    miss = want[~idwant.isin(idhave)]

    return miss


def compile(rootPath, sim_list, rows, mods, ground_material, fileName=None):
    
    # -- create the empty dataframe
    midx_names = ['datetime','row','mod','wavelength']
    blank = [[],[],[],[]]
    midx = pd.MultiIndex(names=midx_names,
                         codes=blank,
                         levels=blank)
    cols = ['Wm2Front','Wm2Back']
    outDF = pd.DataFrame(index=midx, columns=cols)
    outDF[cols].astype('object')

    # -- save this to verify result list
    dates = pd.to_datetime(sim_list.datetime,format='%Y-%m-%d_%H00')
    year = dates[0].year
    month = dates[0].month
    
    # -- read in the results
    tpath = os.path.join(rootPath,'results',ground_material)
    days, dayList = _buffer(tpath)
    print(days)

    for d in days:
        dpath = os.path.join(tpath,d)
        hours, hourList = _buffer(dpath)
        print(hours)
        for h in hours:
            hpath = os.path.join(dpath,h)
            waves, waveList = _buffer(hpath)
            print(waves)
            for w in waves:
                # -- read file
                resPath = os.path.join(hpath,w,'results')
                files, fileList = _buffer(resPath)
                print(files)
                files = next(os.walk(resPath))[2]
                #print(files)
                for file in files:
                    rpath = os.path.join(resPath,file)
                    tdf = pd.read_csv(rpath)
                    print(tdf)
                    
                    # -- scrape indexing
                    day = dayList[days.index(d)]
                    hour = hourList[hours.index(h)]
                    wave = waveList[waves.index(w)]
                    tdate = f'{year}-{month:02}-{day:02}_{hour:02}00'
                    r,m = _rowmod(file)
                    
                    # -- NOTE
                    # check ray contact

                    # -- copy data
                    irrF = tdf['Wm2Front'].values
                    irrB = tdf['Wm2Back'].values
                    outDF.loc[(tdate,r,m,wave),:] = [np.nan,np.nan]
                    outDF.loc[(tdate,r,m,wave),:] = [irrF,irrB]

    # -- save the data
    if not fileName:
        fileName = f'compiled_results.csv'
    filePath = os.path.join(rootPath,fileName)
    outDF.to_csv(filePath)
    print('Results saved.')
    
    # -- verify result list
    missing = _checkForMissing(outDF.index,sim_list,rows,mods)
    if missing is not None:
        missPath = os.path.join(rootPath,f'missing_results.csv')
        missing.to_csv(missPath, index=False)
        print('logged.')
        return None
    else:
        print('done.')
        return None


In [11]:
compile(rootPath=rootPath, sim_list=sim_list, rows=rows, mods=mods, ground_material=ground_material,fileName='compiled_results_mp.csv')


['d_02']
['h_10', 'h_11']
['w_0300', 'w_0325', 'w_0350', 'w_0375', 'w_0400']
[]
       x         y         z     rearZ                        mattype  \
0 -0.972 -5.079202  1.181518  1.163652  a3.0.a2.1.0.cellPVmodule.6457   
1 -0.972 -4.892423  1.315732  1.297866  a3.0.a2.2.0.cellPVmodule.6457   
2 -0.972 -4.705644  1.449947  1.432081  a3.0.a2.4.0.cellPVmodule.6457   
3 -0.972 -4.518865  1.584161  1.566295  a3.0.a2.5.0.cellPVmodule.6457   
4 -0.972 -4.332086  1.718375  1.700510  a3.0.a2.7.0.cellPVmodule.6457   
5 -0.972 -4.145306  1.852590  1.834724  a3.0.a2.8.0.cellPVmodule.6457   

                         rearMat  Wm2Front   Wm2Back  Back/FrontRatio  
0  a3.0.a2.1.0.cellPVmodule.2310  0.002380  0.000103         0.030608  
1  a3.0.a2.2.0.cellPVmodule.2310  0.002382  0.000107         0.031526  
2  a3.0.a2.4.0.cellPVmodule.2310  0.002384  0.000111         0.032941  
3  a3.0.a2.5.0.cellPVmodule.2310  0.002387  0.000118         0.034816  
4  a3.0.a2.7.0.cellPVmodule.2310  0.002290  0.00